I used LightGBM as a first attempt to submit.
Not much EDA or Feature engineering, I only did encode the categoricals.
I think there are so many things for improvment, as I am just new to Kaggle.
Hope you guys like it, and please upvote it!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# EDA

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
delete_columns = ['id']
train.drop(delete_columns, axis=1, inplace=True)
test.drop(delete_columns, axis=1, inplace=True)

In [ ]:
train.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

plt.figure(figsize=(10,5))
sns.histplot(train['target'], color='slategray', stat='frequency');

In [ ]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['target'], plot=plt)
plt.show()

The target variable is right skewed. As (linear) models love normally distributed data , we need to transform this variable and make it more normally distributed.

In [ ]:
len(train[train['target'] < 4])

In [ ]:
to_drop = train[train['target'] < 4].index
train.drop(to_drop, inplace=True)
train = train.reset_index()
train.info()

plt.figure(figsize=(10,5))
sns.histplot(train['target'], color='slategray', stat='frequency');

In [ ]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['target'], plot=plt)
plt.show()

It's look better

In [ ]:
delete_columns = ['index']
train.drop(delete_columns, axis=1, inplace=True)

In [ ]:
train.info()

In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(train, title='Pandas Profiling Report')

In [ ]:
profile

In [ ]:
# drop non-infarmative features: cat0, cat4, cat6, cat7
non_infarmative_features = ['cat0', 'cat4', 'cat6', 'cat7']
train.drop(non_infarmative_features, axis=1, inplace=True)
test.drop(non_infarmative_features, axis=1, inplace=True)

In [ ]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values)+list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)
        
display(train.head())

In [ ]:
VAL_SIZE          = 0.33   # 33%
N_FOLDS           = 5

# RANDOM_SEED
RANDOM_SEED       = 42

# LightGBM


In [ ]:
y_train = train['target']
X_train = train.drop('target', axis = 1)
X_test = test

In [ ]:
y_train.shape, X_train.shape, X_test.shape

In [ ]:
X_train.info()

In [ ]:
categorical_features = ['cat1', 'cat2', 'cat3', 'cat5', 'cat8', 'cat9']

In [ ]:
y_preds = []
models = []
oof_train = np.zeros(len(X_train))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

params = {
    'metric': 'rmse', 
    'random_state': 42,
    'n_estimators': 20000,
    'reg_alpha': 4.739177609135503,
    'reg_lambda': 0.023314865020550128,
    'colsample_bytree': 0.2885575936307841,
    'subsample': 0.9541423485889546,
    'learning_rate': 0.02211501295658431,
    'max_depth': 18,
    'num_leaves': 305,
    'min_child_samples': 118,
    'min_data_per_groups': 89
}

y_preds = 0

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):

    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)
    
    model = lgb.train(params, lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      verbose_eval = 200,
                      num_boost_round = 20000,
                      early_stopping_rounds=1000)
    
    oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_preds += y_pred/5
    models.append(model)

In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

In [ ]:
y_preds_lgb = pd.DataFrame(y_preds)

In [ ]:
# VERSION = 8

In [ ]:
# sample_sub['target'] = y_preds_lgb
# sample_sub.to_csv(f'submission_LightGBM_{VERSION}.csv', index=False)

# CatBoost

In [ ]:
from catboost import Pool, CatBoostRegressor, cv
from catboost import CatBoostClassifier

In [ ]:
# CATBOOST
ITERATIONS        = 3000
LR                = 0.01
MAX_DEPTH         = 10
N_ESTIMATORS      = 3000
L2_LEAF_REG       = 3.5

In [ ]:
X = train.drop(['target'], axis=1,)
y = train.target.values
X_sub = test.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
cat_features_ids = np.where(X_train.dtypes == object)[0].tolist()
cat_features_ids

In [ ]:
ctb = CatBoostRegressor(learning_rate=LR,
                          max_depth=MAX_DEPTH,
                          n_estimators=N_ESTIMATORS,
                          l2_leaf_reg=L2_LEAF_REG,
                          random_seed=RANDOM_SEED,
                          eval_metric='RMSE',
                          custom_metric=['R2', 'MAE']
                         )
ctb.fit(X_train, y_train,
          eval_set=(X_test, y_test),
          verbose_eval=100,
          cat_features=categorical_features,
          use_best_model=True,
          plot=True
         )

In [ ]:
ctb.save_model('catboost_single_model_baseline.ctb')

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

idx = np.argsort(ctb.feature_importances_)

plt.figure(figsize=(17,8))

sns.barplot(x=ctb.feature_importances_[idx], y=np.array(ctb.feature_names_)[idx])

In [ ]:
predict_submission_ctb = ctb.predict(X_sub)
predict_submission_ctb

In [ ]:
# VERSION = 9

In [ ]:
# sample_sub['target'] = predict_submission_ctb
# sample_sub.to_csv(f'submission_CatBoost_v{VERSION}.csv', index=False)
# sample_sub.head(10)

# XGBoost

In [ ]:
X = train.drop(['target'], axis=1,)
y = train.target.values
X_sub = test.copy()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_params = {
    'colsample_bytree':0.4603,
    'gamma':0.0468,
    'learning_rate':0.05,
    'max_depth':3,
    'min_child_weight':1.7817,
    'n_estimators':2200,
    'reg_alpha':0.4640,
    'reg_lambda':0.8571,
    'subsample':0.5213,
    'nthread':-1,
    'eval_metric':'rmse',
    'seed':RANDOM_SEED,
#     'tree_method':'gpu_hist',
#     'gpu_id':0
}

In [ ]:
xgb = XGBRegressor(**xgb_params)
xgb.fit(X_train, y_train)

In [ ]:
y_pred_xgb = xgb.predict(X_test)


In [ ]:
from sklearn import metrics

def RMSE(y, y_pred):
    
    MSE = metrics.mean_squared_error(y, y_pred)
    print(f'MSE = {MSE:.4f}')

    RMSE = np.sqrt(MSE)
    print(f'RMSE = {RMSE:.4f}')

    print('-------')

In [ ]:
rmse = RMSE(y_test, y_pred_xgb)
rmse

In [ ]:
predict_submission_xgb = xgb.predict(X_sub)
predict_submission_xgb

In [ ]:
# VERSION = 3

In [ ]:
# sample_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
# sample_sub['target'] = predict_submission_xgb
# sample_sub.to_csv(f'submission_XGBoost_v{VERSION}.csv', index=False)
# sample_sub.head(10)

# Сравнение features between CatBoost and XGBoost

https://www.youtube.com/watch?v=5Al4pXdJmAY&ab_channel=DataStartConference

In [ ]:
xgb_results = sorted(zip(map(lambda x: round(x, 4), xgb.feature_importances_), X), reverse=True)
ctb_results = sorted(zip(map(lambda x: round(x, 4), ctb.feature_importances_), X), reverse=True)

pd.DataFrame(data = [[(m, v) for v, m in xgb_results],
                     [(m, v) for v, m in ctb_results]],
             index=['xgb','ctb']).T.head(5)

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,8))

ax1.set_xlabel('Порядок параметра')
ax1.set_ylabel('Веса Forest&Tree')

# ось 1
ax1.plot([v for v,m in xgb_results][:100], 'b')

ax2 = ax1.twinx() #добавление оси

ax2.set_ylabel('Веса CatBoost')
ax2.plot([v for v,m in ctb_results][:100], 'y')
fig.tight_layout()
plt.show()


In [ ]:
res_dict = dict()

for i in range(20):
        for mod in ([m for v,m in xgb_results],
                    [m for v,m in ctb_results]):
            if mod[i] in res_dict:
                res_dict[mod[i]] += 1
            else:
                res_dict[mod[i]] = 1

resulted = sorted(zip(map(lambda x: x, res_dict.values()), res_dict.keys()), reverse=True)
resulted[:10]

In [ ]:
for_use_vars = [k for v,k in resulted]
print('В выборке: ', len(for_use_vars))

In [ ]:
zeros = []
for k in [k for v,k in resulted]:
    for model in (xgb_results, ctb_results):
        if k in [m for v,m in model[:100]]:
            if [v for v,m in model][list([m for v,m in model][:100]).index(k)] == 0:
                zeros.append(k)

print('В выборке: ', len(zeros))


must_use = set(for_use_vars) - set(zeros)
print('Уже более подходящее число: ', len(must_use))

In [ ]:
plt.figure(figsize=(20,20))

sns.heatmap(X[X.columns[[list(X.columns).index(m) for m in must_use]]].corr(), annot = True, cmap = 'Blues')

# Stacking

In [ ]:
X = train.drop(['target'], axis=1,)
y = train.target.values
X_sub = test.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
y_train.shape, X_train.shape, X_test.shape

In [ ]:
from sklearn import metrics

def RMSE(y, y_pred):
    
    MSE = metrics.mean_squared_error(y, y_pred)
    print(f'MSE = {MSE:.4f}')

    RMSE = np.sqrt(MSE)
    print(f'RMSE = {RMSE:.4f}')

    print('-------')

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from catboost import Pool, CatBoostRegressor, cv
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import lightgbm as lgb

xgb_params = {
    'colsample_bytree':0.4603,
    'gamma':0.0468,
    'learning_rate':0.05,
    'max_depth':3,
    'min_child_weight':1.7817,
    'n_estimators':2200,
    'reg_alpha':0.4640,
    'reg_lambda':0.8571,
    'subsample':0.5213,
    'nthread':-1,
    'eval_metric':'rmse',
    'seed':RANDOM_SEED,
#     'tree_method':'gpu_hist',
#     'gpu_id':0
}

In [ ]:
xgb = XGBRegressor(**xgb_params)
rfr = RandomForestRegressor(n_estimators=100, max_depth=6, n_jobs=-1)

from sklearn.linear_model import LinearRegression
from sklearn import metrics

lr = LinearRegression()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(metrics.mean_squared_error(y, y_pred))

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (xgb, rfr),
                                                 meta_model = lr)

In [ ]:
stacked_averaged_models.fit(X_train.values, y_train)

In [ ]:
stacked_train_pred = stacked_averaged_models.predict(X_train.values)
print(rmsle(y_train, stacked_train_pred))

In [ ]:
stacked_pred = stacked_averaged_models.predict(X_sub.values)

In [ ]:
VERSION = 5

In [ ]:
df_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
df_sub['target'] = stacked_pred
df_sub.to_csv(f'submission_stacking_v{VERSION}.csv', index=False)
df_sub.head(10)

In [ ]:
xgb.fit(X_train, y_train)
xgb_train_pred = xgb.predict(X_train)
xgb_pred = np.expm1(xgb.predict(X_test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
xgb_pred_stacked = xgb.predict(X_sub)
xgb_pred_stacked

In [ ]:
VERSION = 7

In [ ]:
df_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
df_sub['target'] = xgb_pred_stacked
df_sub.to_csv(f'submission_stacking_v{VERSION}.csv', index=False)
df_sub.head(10)

# Averages

In [ ]:
ensemble = stacked_pred*0.2 + predict_submission_xgb*0.2 + predict_submission_ctb*0.3 + y_preds_lgb.values*0.3

df_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
df_sub['target'] = ensemble
df_sub.to_csv(f'submission_average.csv', index=False)
df_sub.head(10)

In [ ]:
print(len(predict_submission_ctb))
print(len(predict_submission_xgb))
print(len(y_preds_lgb))

In [ ]:
sub_predict_xgb = pd.read_csv('./submission_XGBoost_v3.csv')
sub_predict_catboost = pd.read_csv('./submission_CatBoost_v3.csv')
sub_predict_lgbm = pd.read_csv('./submission_LightGBM_8.csv')

df_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')
blend_sub_predict = (sub_predict_xgb['target'] + sub_predict_catboost['target'] + sub_predict_lgbm['target']) / 3
df_sub['target'] = blend_sub_predict
df_sub.to_csv(f'submission_average.csv', index=False)
df_sub.head(10)